In [94]:
# Packages
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import json
import time
from datetime import datetime

In [13]:
# Chrome options
options = uc.ChromeOptions()
options.page_load_strategy = "eager"
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-notifications")
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-extensions")
options.add_argument("--blink-settings=imagesEnabled=false")
options.add_argument("--disable-gpu")
options.add_argument("--lang=en-US")
options.add_argument("--disable-background-timer-throttling")
options.add_argument("--disable-backgrounding-occluded-windows")
options.add_argument("--disable-renderer-backgrounding")

# Driver
driver = uc.Chrome(options=options)
driver.set_page_load_timeout(30)

wait = WebDriverWait(driver, 20)

print("WebDriver initialized.")

# CHANGED: lock in the params you’re actually using
base_url = "https://www.metacritic.com/browse/game/"
params = "releaseYearMin=1958&releaseYearMax=2025"

driver.get(f"{base_url}?{params}&page=1")

url_list = []
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "c-productListings_grid")))

# How many pages to scrape?
pages = driver.find_elements(By.CLASS_NAME, "c-navigationPagination_item")
total_pages = int(pages[-2].text)
print(f"Total pages to scrape: {total_pages}")

for i in range(1, 5 + 1):
    time.sleep(1)

    driver.get(f"{base_url}?{params}&page={i}")

    # Wait for listings
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "c-productListings")))

    block = driver.find_element(By.CLASS_NAME, "c-productListings")
    items = block.find_elements(By.TAG_NAME, "a")

    for a in items:
        link = a.get_attribute("href")
        if link and "/game/" in link:
            url_list.append(link)

    print(f"Page {i}/{total_pages} | Total links scraped: {len(url_list)}")

print(f'Scraping urls is over, total games found: {len(url_list)}')

print("Calculating total scrape time...")
test_urls = url_list[:10]
times = []
# scrape 5 to make a calculation on average time of scraping.
for url in test_urls:
    # Start timer
    start = time.perf_counter()
    
    driver.get(url)

    # Get info on page
    score_container = driver.find_element(by.CLASS_NAME, 'c-productHero_score-container')
    game_title = score_container.find_element(By.CLASS_NAME, 'c-productHero_title').text
    print(game_title)

    platform = 

    metascore = 

    user_score = 

    

    # Stop timer
    end = time.perf_counter()

    
    times.append(end - start)

average_time = sum(times) / len(times)

print(
    f"Average time per URL: {average_time:.2f} seconds. "
    f"Total expected scraping time = approximately {(average_time * len(url_list)) / 60:.0f} minutes"
)


for url in url_list[:5]:
    driver.get(url)
    print(f'Scraping: {url}')

driver.quit()

print("Webdriver closed. Finished full scraping process succesfully!")

WebDriver initialized.
Total pages to scrape: 584
Page 1/584 | Total links scraped: 24
Page 2/584 | Total links scraped: 48
Page 3/584 | Total links scraped: 72
Page 4/584 | Total links scraped: 96
Page 5/584 | Total links scraped: 120
Scraping urls is over, total games found: 120
Calculating total scrape time...
Average time per URL: 1.11 seconds. Total expected scraping time = approx2 minutes
Scraping: https://www.metacritic.com/game/the-legend-of-zelda-ocarina-of-time/
Scraping: https://www.metacritic.com/game/soulcalibur/
Scraping: https://www.metacritic.com/game/grand-theft-auto-iv/
Scraping: https://www.metacritic.com/game/super-mario-galaxy/
Scraping: https://www.metacritic.com/game/super-mario-galaxy-2/
Scraping: https://www.metacritic.com/game/the-legend-of-zelda-breath-of-the-wild/
Scraping: https://www.metacritic.com/game/perfect-dark-2000/
Scraping: https://www.metacritic.com/game/tony-hawks-pro-skater-3/
Scraping: https://www.metacritic.com/game/red-dead-redemption-2/
Scra

In [104]:
options = uc.ChromeOptions()
options.page_load_strategy = "eager"
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-notifications")
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-extensions")
options.add_argument("--blink-settings=imagesEnabled=false")
options.add_argument("--disable-gpu")
options.add_argument("--lang=en-US")
options.add_argument("--disable-background-timer-throttling")
options.add_argument("--disable-backgrounding-occluded-windows")
options.add_argument("--disable-renderer-backgrounding")

# Driver
driver = uc.Chrome(options=options)
driver.set_page_load_timeout(30)

wait = WebDriverWait(driver, 20)

for url in test_urls[:2]:
    # Start timer
    start = time.perf_counter()
    
    driver.get(url)

    # Get info on page
    score_container = driver.find_element(By.CLASS_NAME, 'c-productHero_score-container')

    # Get details container
    details_container = driver.find_element(By.CLASS_NAME, 'c-pageProductionDetails')

    # Game title
    game_title = score_container.find_element(By.CLASS_NAME, 'c-productHero_title').text

    # Game platform
    platform_list = []
    platforms_block = details_container.find_element(By.CLASS_NAME, 'c-gameDetails_Platforms')
    platforms = platforms_block.find_elements(By.CLASS_NAME, 'c-gameDetails_listItem')
    for platform in platforms:
        platform_list.append(platform.text)    

    # Release data as datetime object
    release_date_str = score_container.find_element(By.CLASS_NAME, 'u-text-uppercase').text
    release_date = datetime.strptime(release_date_str, "%b %d, %Y").date()

    # Developer
    developer_block = details_container.find_element(By.CLASS_NAME, 'c-gameDetails_Developer')
    developer = developer_block.find_element(By.CLASS_NAME, 'c-gameDetails_listItem').text

    # Publisher/Distributor
    publisher_block = details_container.find_element(By.CLASS_NAME, 'c-gameDetails_Distributor')
    publisher = publisher_block.find_element(By.CLASS_NAME, 'u-text-underline').text
    
    # Game genre
    genre = details_container.find_element(By.CLASS_NAME, 'c-genreList').text

    # Metascore from critics
    metascore = score_container.find_element(By.CLASS_NAME, 'c-productScoreInfo_scoreNumber').text
    
#    total_crit_reviews =
#    crit_pos = 
#    crit_mix = 
#    crit_neg = 

    # Userscore from users
    user_score = score_container.find_element(By.CLASS_NAME, 'c-siteReviewScore_background-user').text
    
#    total_user_reviews = 
#    user_pos = 
#    user_mix = 
#    user_neg = 

    gamestat_row = driver.find_element(By.CLASS_NAME, 'gameStats_row')
    view_more_btn = gamestat_row.find_element(By.CLASS_NAME, 'c-globalButton_container')
    driver.execute_script("arguments[0].click();", view_more_btn)


    # Difficultie votes
    diff_stats_block = driver.find_element(By.CLASS_NAME, 'c-gameStatsWidgetModal_content')
    diff_stat_elements = diff_stats_block.find_elements(By.CLASS_NAME, 'c-globalChart-legendElement')

#    total_diff_votes =
    diff_unforgiving = float(diff_stat_elements[0].text.split()[-1].replace('%', ''))
    diff_tough = float(diff_stat_elements[1].text.split()[-1].replace('%', ''))
    diff_justright = float(diff_stat_elements[2].text.split()[-1].replace('%', ''))
    diff_easy = float(diff_stat_elements[3].text.split()[-1].replace('%', ''))
    diff_simple = float(diff_stat_elements[4].text.split()[-1].replace('%', ''))
    print(diff_justright)

    # Playtime votes
    diff_block = driver.find_element(By.CLASS_NAME, 'c-globalModal_content')
    buttons = diff_block.find_elements(By.CLASS_NAME, 'c-globalHeader_menu_item')
    playtime_btn = buttons[1]
    driver.execute_script("arguments[0].click();", playtime_btn)
    time.sleep(2)
    
#    total_playtime_votes = 
#    time_under1 =
#    time_1 = 
#    time_2 = 
#    time_4 = 
#    time_8 = 
#    time_12 = 
#    time_20 = 
#    time_40 = 
#    time_60 = 
#    time_over80 =

    
    # Completion votes
    completion_btn =buttons[2]
    driver.execute_script("arguments[0].click();", completion_btn)
    time.sleep(2)
#    total_completion_votes = 
#    comp_triedit = 
#    comp_playedit =
#    comp_halfway = 
#    comp_beatit = 
#    comp_conqueredit =


driver.quit()

75.35
80.66


In [15]:
test_urls[:2]

['https://www.metacritic.com/game/the-legend-of-zelda-ocarina-of-time/',
 'https://www.metacritic.com/game/soulcalibur/']

In [34]:
set(url_list)
len(url_list)

14010

In [57]:
url_list

['https://www.metacritic.com/game/the-legend-of-zelda-ocarina-of-time/',
 'https://www.metacritic.com/game/soulcalibur/',
 'https://www.metacritic.com/game/grand-theft-auto-iv/',
 'https://www.metacritic.com/game/super-mario-galaxy/',
 'https://www.metacritic.com/game/super-mario-galaxy-2/',
 'https://www.metacritic.com/game/the-legend-of-zelda-breath-of-the-wild/',
 'https://www.metacritic.com/game/perfect-dark-2000/',
 'https://www.metacritic.com/game/tony-hawks-pro-skater-3/',
 'https://www.metacritic.com/game/red-dead-redemption-2/',
 'https://www.metacritic.com/game/grand-theft-auto-v/',
 'https://www.metacritic.com/game/metroid-prime/',
 'https://www.metacritic.com/game/grand-theft-auto-iii/',
 'https://www.metacritic.com/game/super-mario-odyssey/',
 'https://www.metacritic.com/game/halo-combat-evolved/',
 'https://www.metacritic.com/game/nfl-2k1/',
 'https://www.metacritic.com/game/half-life-2/',
 'https://www.metacritic.com/game/bioshock/',
 'https://www.metacritic.com/game/gol

In [ ]:
#